# Глубокое обучение

# Лекция 2: Обучение искуственных нейронных сетей в PyTorch

__Автор: Сергей Вячеславович Макрушин__ e-mail: s-makrushin@yandex.ru 

Финансовый универсиет, 2023 г. 

V 0.8 20.09.2023

При подготовке лекции использованы материалы:
* Николенко, Кадурин, Архангельская "Глубокое обучение. Погружение в мир нейронных сетей"
* Документация PyTorch: https://pytorch.org/docs/stable/index.html

## Разделы: <a class="anchor" id="разделы"></a>
* [Современные методы обучения нейронной сети и обратное распространение ошибки](#современные-методы)
* [Обучение модели нейронной сети, алгоритм обратного распространения ошибки](#обратное-распространение)
    * [Задача обучения модели нейронной сети](#проблема-обучения)
    * [Проблема поиска градиента](#проблема-поиска)
* [Дифференцируемое программирование и реализация обратного распространения ошибки](#дифференцируемое)
    * [Автоматическое дифференциирование в PyTorch](#автоматическое-PyTorch)

-

* [к оглавлению](#разделы)

In [3]:
# загружаем стиль для оформления презентации
from IPython.display import HTML
from urllib.request import urlopen
html = urlopen("file:./lec_v2.css")
HTML(html.read().decode('utf-8'))

## Современные методы обучения нейронной сети и обратное распространение ошибки <a class="anchor" id="современные-методы"></a>
* [к оглавлению](#разделы)

----
### Применение тензоров:  прямое распространение сигналов и оценка ошибки

__Постановка задачи__ 

* У нас есть набор данных $D$, состоящий из пар $(\pmb{x}, \pmb{y})$, где $\pmb{x}$ - признаки, а $\pmb{y}$ - правильный ответ. 
* Модель сети $f_L$, имеющей $L$ слоев с весами $\pmb{\theta}$ (совокупность весов нейронов из всех слоев) на этих данных делает некоторые предсказания $\hat{\pmb{y}} = f_L(\pmb{x}, \pmb{\theta})$
* Задана функция ошибки $E$, которую можно подсчитать на каждом примере: $E(f_L(\pmb{x}, \pmb{\theta}), \pmb{y})$ (например, это может быть квадрат или модуль отклонения $\hat{\pmb{y}}$ от $\pmb{y}$ в случае регрессии или перекрестная энтропия в случае классификации)
* Тогда суммарная ошибка на наборе данных $D$ будет функцией от параметров модели: $E(\pmb{\theta})$ и определяется как $E(\pmb{\theta})=\sum_{(\pmb{x}, \pmb{y}) \in D} E(f_L(\pmb{x}, \pmb{\theta}), \pmb{y})$

<center> 
<img src="./img/main_cycle_p2_v2.png" alt="Прямой проход и оценка ошибки" style="width: 600px;"/><br/>
    <b>Прямой проход и оценка ошибки</b>    
</center> 


__Прямое распространение сигналов__

* Модель нейронной сети это иерархия (она может быть простой и очень сложной) связанных (последовательно применяемых) функций слоев:
    * т.е. модель сети $f_L$ может быть представленна как суперпозиция из $L$ слоев $h^i\text{, }i \in \{1, \ldots, L\}$, каждый из которых параметризуется своими весами $w_i$:
$$f_L(\pmb{x}, \pmb{\theta})=f_L(\pmb{x}, \pmb{w}_1, \ldots, \pmb{w}_L )=h^L(h^{L-1}(\ldots h^1(\pmb{x}, \pmb{w}_1), \ldots, \pmb{w}_{L-1}),\pmb{w}_L)$$

<center> 
<img src="./img/ann_11.png" alt="многослойный перцептрон с двумя скрытыми слоями" style="width: 600px;"/><br/>
    <b><em class="ex"></em> пример модели сети: многослойный перцептрон с двумя скрытыми слоями</b>    
</center> 

* Прямое распространение сигналов по модели (в частности: нейронной сети) реализуется с помощощью __прямого прохода (forward pass)__: входящая информация (вектор $\pmb{x}$) распространяется через сеть $f_L$ с учетом весов связей $\pmb{\theta}$, расчитывается выходной вектор $\hat{\pmb{y}}=f_L(\pmb{x}, \pmb{\theta})$ .
    * Каждый слой нейронной сети - это последовательно применяемая функция слоя, которая рассчитывается при помощи операций с тензорами.

<center> 
<img src="./img/ann_12.png" alt="пример прямого прохода" style="width: 600px;"/><br/>
    <b><em class="ex"></em> пример прямого прохода</b>    
</center> 

* Последовательность операций с тензормаи используется для расчета результата:
    * Прямой проход (forward pass): входящая информация (вектор $\pmb{x}$) распространяется через сеть с учетом весов связей, расчитывается выходной вектор $\hat{\pmb{y}} = f_L(\pmb{x}, \pmb{\theta})= h^L(h^{L-1}(\ldots h^1(\pmb{x}, \pmb{w}_1), \ldots, \pmb{w}_{L-1}),\pmb{w}_L)$
    * Оценки ошибки $E(\hat{\pmb{y}}, \pmb{y})$ на множестве правильных ответов: $\pmb{y}$.

<center> 
<img src="./img/main_cycle_p2_v2.png" alt="Прямой проход и оценка ошибки" style="width: 600px;"/><br/>
    <b>Прямой проход и оценка ошибки</b>    
</center> 

---
## Обучение модели нейронной сети, алгоритм обратного распространения ошибки <a class="anchor" id="обратное-распространение"></a>
* [к оглавлению](#разделы)

### Задача обучения модели нейронной сети <a class="anchor" id="проблема-обучения"></a>
* [к оглавлению](#разделы)

__Проблема обучения модели нейронной сети: общий взгляд__

* <em class="nt"></em> __основная проблема__ это не применение модели к входным данным $\pmb{x}$ и оцнка ошибки на правильных ответах $\pmb{y}$, а __обучение модели__ (опредление наилучших параметров модели $\pmb{\theta}$). 
     * В случае нейронной сети обучение сводится к поиску весов слоев сети $\pmb{\theta}=(\pmb{w}_1, \ldots, \pmb{w}_L)$, которые в совокупности являются параметрами модели $\pmb{\theta}$.

* Формально: цель обучения - найти оптимальное значение параметров $\theta^{*}$, минимизирующих ошибку на обучающией выборке $D$: 
$$\theta^{*} = \arg \underset{\pmb{\theta}}{\min} \ E(\pmb{\theta}) = \arg \underset{\pmb{\theta}}{\min} \ \sum_{(\pmb{x}, \pmb{y}) \in D} E(f_L(\pmb{x}, \pmb{\theta}), \pmb{y})$$
* Т.е. задача обучения сводится к задаче оптимизации.
    * <em class="nt"></em> На самом деле __все сложнее__: хороший результат на $D$ может плохо обобщаться (модель может давать низкое качество на другой выборке из той же генеральной совокупности) - __проблема переобучения__.

<center> 
<img src="./img/main_cycle_p1_v1.png" alt="Приниципиальная логика обучения нейронной сети" style="width: 800px;"/><br/>
    <b>Приниципиальная логика обучения нейронной сети</b>    
</center>

__Прямой проход и оценка ошибки__

* __Прямой проход__ (forward pass): входящая информация (вектор $\pmb{x}$) распространяется через сеть с учетом весов связей, расчитывается выходной вектор $\hat{\pmb{y}} = f_L(\pmb{x}, \pmb{\theta})= h^L(h^{L-1}(\ldots h^1(\pmb{x}, \pmb{w}_1), \ldots, \pmb{w}_{L-1}),\pmb{w}_L)$

<center> 
<img src="./img/ann_12.png" alt="пример прямого прохода" style="width: 300px;"/><br/>
    <b><em class="ex"></em> пример прямого прохода</b>    
</center> 

* __Оценки ошибки__ $E(\hat{\pmb{y}}, \pmb{y})$ на множестве правильных ответов: $\pmb{y}$.

<center> 
<img src="./img/main_cycle_p2_v2.png" alt="Прямой проход и оценка ошибки" style="width: 400px;"/><br/>
    <b>Прямой проход и оценка ошибки в общей логике обучения нейронной сети</b>    
</center> 

__Задача оптимизации__

* Задача: корректировка весов сети (параметров модели $\pmb{\theta}$) на основе информации об ошибке на обучающих примерах $E(\hat{\pmb{y}}, \pmb{y})$.
    * Решение: использовать методы оптимизации, основанные на __методе градиентного спуска__.
    

* __Метод градиентныого спуска__ - метод нахождения локального экстремума (минимума или максимума) функции с помощью движения вдоль градиента. В нашем случае шаг метода градиентного спуска выглядит следующим образом:
$$\pmb{\theta}_t = \pmb{\theta}_{t-1}-\gamma\nabla_\theta E(\pmb{\theta}_{t-1}) = \pmb{\theta}_{t-1}-\gamma \sum_{(\pmb{x}, \pmb{y}) \in D} \nabla_\theta E(f_L(\pmb{x}, \pmb{\theta}), \pmb{y})$$

* <em class="nt"></em> Выполнение на каждом шаге градиентого спуска суммирование по всем $(\pmb{x}, \pmb{y}) \in D$ __обычно слшиком неэффективно__


* Для выпуклых функций __задача локальной оптимизации__ - найти локальный минимум (максимум) автоматически превращается в __задачу глобальной оптимизации__ - найти точку, в которой достигается наименьшее (наибольшее) значение функции, то есть самую низкую (высокую) точку среди всех.
* Оптимизировать веса одного перцептрона - выпуклая задача, но __для большой нейронной сети  целевая функция не является выпуклой__.

<center> 
<img src="./img/ann_15.png" alt="Прямой проход и оценка ошибки" style="width: 500px;"/><br/>
    <b>Пример работы градиентного спуска для функции двух переменных</b>    
</center>

* У нейронных сетей функция ошибки может задавать __очень сложный ландшафт__ с огромным числом локальных максимумов и минимумов. Это свойство необходимо для обеспечения __выразительности нейронных сетей__, позволяющей им решать так много разных задач.


* <em class="nt"></em> для использования методов, основанных на методе градиентного спуска __необходимо знать градиент функции потерь по параметрам модели__: $\nabla_\theta E(f_L(\pmb{x}, \pmb{\theta}), \pmb{y})$. Этот градиент определяет вектор ("направление") изменения параметров.

---
#### Пример: обучение модели линейной регрессии

__Исходные данные:__

In [5]:
import torch
import numpy as np

In [14]:
# Модель линейной регрессии (с несколькими параметрами)
# f = X * w 

# Данные для обучения: 
# принзаки X: рассматривается 4 наблюдения (ось 0) и 2 признака (ось 1):

# вариант исходных данны #1 (2й признак всегда равен 0):
# X = torch.tensor([[1., 0.],
#                   [2., 0.],
#                   [3., 0.],
#                   [4., 0.]], dtype=torch.float32) # Size([4, 2])

# вариант исходных данны #2 (2й признак используется и существенно больше 1го):
X = torch.tensor([[1., 40.],
                  [2., 30.],
                  [3., 20.],
                  [4., 10.]], dtype=torch.float32) # Size([4, 2])

print(f'Матрица X: \n{X}')
print(f'X.size = {X.size()}') 

# истинное значение весов (используется только для получения обучающих правильных ответов):

# вариант #1:
# w_ans = torch.tensor([2., 0.], dtype=torch.float32)

# вариант #2:
w_ans = torch.tensor([2., 1.], dtype=torch.float32)

# Y - приавильные ответы: 
Y = X @ w_ans

torch.set_printoptions(precision=5) # точность вывода на печать значений тензоров
print(f'w true value = {w_ans}, Y = {Y}')

Матрица X: 
tensor([[ 1., 40.],
        [ 2., 30.],
        [ 3., 20.],
        [ 4., 10.]])
X.size = torch.Size([4, 2])
w true value = tensor([2., 1.]), Y = tensor([42., 34., 26., 18.])


__Модель (прямой проход):__

In [15]:
# model (модель, в нашем случае: линейная регрессия)

# изначальное значение весов w
w = torch.tensor([0.0, 0.0], dtype=torch.float32, requires_grad=False)

print(f'w:\n{w}')

# прямое распространение:
def forward(X):
    return X @ w # Size([4])



w:
tensor([0., 0.])


<center> 
<img src="./img/main_cycle_p2_v2.png" alt="Прямой проход и оценка ошибки" style="width: 400px;"/><br/>
    <b>Прямой проход и оценка ошибки в общей логике обучения нейронной сети</b>    
</center> 

__Функция ошибки и рассчет градиента (аналитический):__

In [16]:
# loss = MSE (функция потерь, в нашем слаучае: средняя квадратичная ошибка)
def loss(y, y_pred):
    return ((y_pred - y)**2).mean() # Size([])

# градиент: 
# рассчитан аналитически по модели и функции потерь:
# J = MSE = 1/N * (w*x - y)**2
# dJ/dw = 1/N * 2 * (w*x - y) * x
def gradient(x, y, y_pred):   
#     print(f'''y = {y},
#     y_pred = {y_pred},
#     (2* (y_pred - y)).unsqueeze(1) = {(2* (y_pred - y)).unsqueeze(1)},
#     x = {x},
#     ((2* (y_pred - y)).unsqueeze(1) * x) = {((2* (y_pred - y)).unsqueeze(1) * x)},
#     ((2* (y_pred - y)).unsqueeze(1) * x).mean(dim=0) = {((2* (y_pred - y)).unsqueeze(1) * x).mean(dim=0)}''')
    return ((2* (y_pred - y)).unsqueeze(1) * x).mean(dim=0)    

__Один проход обучения:__

In [17]:
learning_rate = 0.0013

# predict = forward pass
y_pred = forward(X)

# loss
l = loss(Y, y_pred)

# calculate gradients
dw = gradient(X, Y, y_pred)

print(f'epoch BEFORE LEARNING: w = {w}, y_pred = {y_pred}, loss = {l:.8f}\ngradient = {dw}')

# update weights
w -= learning_rate * dw

epoch BEFORE LEARNING: w = tensor([0., 0.]), y_pred = tensor([0., 0., 0., 0.]), loss = 980.00000000
gradient = tensor([ -130., -1700.])


In [18]:
# predict = forward pass
y_pred = forward(X)

# loss
l = loss(Y, y_pred)

print(f'epoch AFTER FIRST STEP: w = {w}, y_pred = {y_pred}, loss = {l:.8f}\ngradient = {dw}')

epoch AFTER FIRST STEP: w = tensor([0.16900, 2.21000]), y_pred = tensor([88.56900, 66.63800, 44.70700, 22.77600]), loss = 901.66833496
gradient = tensor([ -130., -1700.])


__Основной цикл обучения:__

In [22]:
# изначальное значение весов w
w = torch.tensor([0.0, 0.0], dtype=torch.float32, requires_grad=False)

In [23]:
# Training

# вариант #1:
# learning_rate = 0.05
# n_iters = 20 + 1

# вариант #2:
learning_rate = 0.0013
n_iters = 1000 + 1

# основной цикл:
for epoch in range(n_iters):
    # predict = forward pass
    y_pred = forward(X)

    # loss
    l = loss(Y, y_pred)
    
    # calculate gradients
    dw = gradient(X, Y, y_pred)

    # update weights
    w -= learning_rate * dw

    if epoch % 5 == 0:
        print(f'ep {epoch}: w = {w}, y_p = {y_pred}, loss = {l:.8f}')                
#         print(f'ep {epoch}: w = {w}, y_p = {y_pred}, loss = {l:.8f}\ngrad = {dw}')        
#         print(f'ep {epoch:3}: y_p = {y_pred}, loss = {l:.8f} grd = {dw}')

ep 0: w = tensor([0.16900, 2.21000]), y_p = tensor([0., 0., 0., 0.]), loss = 980.00000000
ep 5: w = tensor([0.13813, 0.24422]), y_p = tensor([81.70274, 61.57523, 41.44772, 21.32021]), loss = 646.58898926
ep 10: w = tensor([0.33966, 1.82453]), y_p = tensor([15.22918, 11.70162,  8.17406,  4.64650]), loss = 427.49359131
ep 15: w = tensor([0.34364, 0.53338]), y_p = tensor([68.78387, 52.09386, 35.40386, 18.71385]), loss = 283.42648315
ep 20: w = tensor([0.49880, 1.56850]), y_p = tensor([25.13912, 19.37721, 13.61530,  7.85340]), loss = 188.61242676
ep 25: w = tensor([0.52316, 0.72007]), y_p = tensor([60.23328, 45.87371, 31.51414, 17.15457]), loss = 126.13953400
ep 30: w = tensor([0.64554, 1.39771]), y_p = tensor([31.56791, 24.41181, 17.25570, 10.09960]), loss = 84.91027832
ep 35: w = tensor([0.68103, 0.83984]), y_p = tensor([54.55604, 41.79293, 29.02982, 16.26671]), loss = 57.64197159
ep 40: w = tensor([0.77979, 1.28313]), y_p = tensor([35.72054, 27.71401, 19.70748, 11.70095]), loss = 39.554

ep 370: w = tensor([1.96556, 1.00231]), y_p = tensor([42.05854, 34.00038, 25.94222, 17.88406]), loss = 0.00505164
ep 375: w = tensor([1.96737, 1.00219]), y_p = tensor([42.05547, 34.00037, 25.94528, 17.89019]), loss = 0.00453248
ep 380: w = tensor([1.96910, 1.00207]), y_p = tensor([42.05252, 34.00034, 25.94816, 17.89598]), loss = 0.00406663
ep 385: w = tensor([1.97073, 1.00196]), y_p = tensor([42.04977, 34.00034, 25.95090, 17.90147]), loss = 0.00364873
ep 390: w = tensor([1.97227, 1.00186]), y_p = tensor([42.04712, 34.00031, 25.95349, 17.90667]), loss = 0.00327375
ep 395: w = tensor([1.97374, 1.00176]), y_p = tensor([42.04465, 34.00030, 25.95595, 17.91160]), loss = 0.00293732
ep 400: w = tensor([1.97512, 1.00167]), y_p = tensor([42.04229, 34.00028, 25.95827, 17.91626]), loss = 0.00263548
ep 405: w = tensor([1.97643, 1.00158]), y_p = tensor([42.04005, 34.00026, 25.96047, 17.92068]), loss = 0.00236459
ep 410: w = tensor([1.97768, 1.00150]), y_p = tensor([42.03794, 34.00025, 25.96256, 17.9

ep 740: w = tensor([1.99938, 1.00004]), y_p = tensor([42.00105, 34.00000, 25.99895, 17.99790]), loss = 0.00000166
ep 745: w = tensor([1.99941, 1.00004]), y_p = tensor([42.00101, 34.00001, 25.99901, 17.99801]), loss = 0.00000149
ep 750: w = tensor([1.99944, 1.00004]), y_p = tensor([42.00096, 34.00001, 25.99907, 17.99812]), loss = 0.00000133
ep 755: w = tensor([1.99947, 1.00004]), y_p = tensor([42.00090, 34.00000, 25.99911, 17.99822]), loss = 0.00000119
ep 760: w = tensor([1.99950, 1.00003]), y_p = tensor([42.00086, 34.00001, 25.99916, 17.99831]), loss = 0.00000107
ep 765: w = tensor([1.99952, 1.00003]), y_p = tensor([42.00081, 34.00000, 25.99920, 17.99840]), loss = 0.00000096
ep 770: w = tensor([1.99955, 1.00003]), y_p = tensor([42.00076, 34.00000, 25.99924, 17.99848]), loss = 0.00000086
ep 775: w = tensor([1.99957, 1.00003]), y_p = tensor([42.00073, 34.00001, 25.99929, 17.99857]), loss = 0.00000077
ep 780: w = tensor([1.99960, 1.00003]), y_p = tensor([42.00068, 34.00000, 25.99932, 17.9

### Проблема поиска градиента <a class="anchor" id="проблема-поиска"></a>
* [к оглавлению](#разделы)

* <em class="qs"></em> Проблема: как найти градиент для нейронной сети: $\nabla_\theta E(f_L(\pmb{x}, \pmb{\theta}), \pmb{y})$?

<center> 
<img src="./img/main_cycle_p3_v1.png" alt="Прямой проход и оценка ошибки" style="width: 400px;"/><br/>
    <b>Проблема поиска градиента в общей логике обучения нейронной сети</b>    
</center> 

* Для решения этой задачи и используется __алгоритм обратного распространения ошибки__ (backpropagation). Суть алгоритма:
    * рассчитывается ошибка между выходным вектором сети $\hat{\pmb{y}}$ и правильным ответом обучающего примера $\pmb{y}$
    * ошибка распростаняется от результата к источнику (в обратную сторону) для корректировки весов

<center> 
<img src="./img/ann_13.png" alt="Пример обратного распространения ошибки" style="width: 400px;"/><br/>
    <b><em class="ex"></em>Пример обратного распространения ошибки</b>    
</center>

__Рассчет градиента суперпозиции двух функций нескольких переменных__

* Сначала рассмотрим подзадачу: как рассчитать градиент для $f_L(\mathbf{x}, \mathbf{w}_1, \ldots, \mathbf{w}_L )=h^L(h^{L-1}(\ldots h^1(\mathbf{x}, \mathbf{w}_1), \ldots, \mathbf{w}_{L-1}),\mathbf{w}_L)$? 

* Для этого нам нужно будет __рассчитывать градиент суперпозиции (сложной функции)__ состоящей из последовательного применения функций слоев $h^i$.

* Вспомним, как рассчитать производную (градиент) суперпозиции нескольких функций.
    * Пусть $z=f(y)$, $y=g(x)$
    * Тогда производная суперпозиции функций (правило дифференцирования сложной функции (chain rule)): $\frac{\mathrm{d} z}{\mathrm{d} x}=\frac{\mathrm{d} z}{\mathrm{d} y}\frac{\mathrm{d} y}{\mathrm{d} x}$
    * Если $\mathbf{x} \in \mathbb{R}^n$, $\mathbf{y} \in \mathbb{R}^m$, а $\mathbf{z} \in \mathbb{R}$, то: $\frac{\partial z }{\partial x_i} = \sum_j \frac{\partial z}{\partial y_j} \frac{\partial y_j}{\partial x_i}$

<center> 
    
__Примеры рассчета градиента суперпозиции двух функций нескольких переменных:__

<img src="./img/ann_18.png" alt="Примеры иерархий в нейронных сетях" style="width: 500px;"/>
</center>
Т.е. нам нужны градиенты по всем возможным путям (рассмотренным в обработном порядке) завимиостей переменных.

Запись этой же задачи в векторной нотации: 
* $\frac{\mathrm{d} z}{\mathrm{d} \mathbf{x}} = \nabla_x (z)= \begin{pmatrix}
    \dfrac{\partial z}{\partial x_1} \\ \cdots \\ \dfrac{\partial z}{\partial x_n} \end{pmatrix}=\left ( \frac{\mathrm{d} \mathbf{y}}{\mathrm{d} \mathbf{x}} \right )^T \cdot \nabla_y (z) = J(\mathbf{y}(\mathbf{x}))^T \cdot \nabla_y (z)= J(\mathbf{y}(\mathbf{x}))^T \cdot \begin{pmatrix}
    \dfrac{\partial z}{\partial y_1} \\ \cdots \\ \dfrac{\partial z}{\partial y_m} \end{pmatrix}$    
* Где $J$ это Якобиан: $$J(\mathbf{y}(\mathbf{x})) = \begin{pmatrix}
    \dfrac{\partial y_1}{\partial x_1} & \cdots & \dfrac{\partial y_1}{\partial x_n}\\
    \vdots & \ddots & \vdots\\
    \dfrac{\partial y_m}{\partial x_1} & \cdots & \dfrac{\partial y_m}{\partial x_n} \end{pmatrix} $$ 

__Задача поиска градиента: $\nabla_\theta E(f_L(\mathbf{x}, \mathbf{\theta}), \mathbf{y})$__

* Перейдем от $f_L$ к последовательному рассчету функций слоев $h^i$:
$$\nabla_\theta E(f_L(\mathbf{x}, \mathbf{\theta}), \mathbf{y})=\nabla_{\mathbf{w}_i} E(f_L(\mathbf{x}, \mathbf{w}_1, \ldots, \mathbf{w}_L ), \mathbf{y})=\nabla_{\mathbf{w}_i} E(h^L(h^{L-1}(\ldots h^1(\mathbf{x}, \mathbf{w}_1), \ldots, \mathbf{w}_{L-1}),\mathbf{w}_L), \mathbf{y})$$

* Обозначим через $\mathbf{a}^l$ результат рассчета функции активации на слое $l$: $\mathbf{a}^l=h^l(\mathbf{x}_l,\mathbf{w}_l)$. Тогда: $\mathbf{x}_{l+1}=\mathbf{a}_l$ (вход следущего слоя является результатом рассчета функции активации предыдущего слоя)

* Тогда можно записать: $\nabla_\theta E(f_L(\mathbf{x}, \mathbf{\theta}), \mathbf{y})=\nabla_\theta E(\mathbf{a}^L, \mathbf{y})$. Функция потерь $E(\mathbf{a}^L, \mathbf{y})$ зависит от $\mathbf{a}^L$, $\mathbf{a}^L$ от $\mathbf{a}^{L-1}$, ..., $\mathbf{a}^{l+1}$ от $\mathbf{a}^{l}$

* Исходя из этого представления можно градиенты весов $l$-го слоя можно записать как: 
$$\dfrac{\partial E}{\partial \mathbf{w}_l}=\color{blue}{ \dfrac{\partial E}{\partial \mathbf{a}_L} \cdot \dfrac{\partial \mathbf{a}_L}{\partial \mathbf{a}_{L-1}} \cdot \cdots \cdot \dfrac{\partial \mathbf{a}_{l+1}}{\partial \mathbf{a}_{l}}} \cdot \color{red}{ \dfrac{\partial \mathbf{a}_{l}}{\partial \mathbf{w}_{l}}}$$

* Произведение всех сомножетелей кроме последнего является градиентом функции потерь по результатам рассчета функции активации слоя $l$:
$$\color{blue} {\dfrac{\partial E}{\partial \mathbf{a}_L} \cdot \dfrac{\partial \mathbf{a}_L}{\partial \mathbf{a}_{L-1}} \cdot \cdots \cdot \dfrac{\partial \mathbf{a}_{l+1}}{\partial \mathbf{a}_{l}}} = \color{blue} {\dfrac{\partial E}{\partial \mathbf{a}_l}}$$

* Тогда:
$$\dfrac{\partial E}{\partial \mathbf{w}_l}=\left ( \color{red}{ \dfrac{\partial \mathbf{a}_{l}}{\partial \mathbf{w}_{l}}} \right )^T \cdot \color{blue}{\dfrac{\partial E}{\partial \mathbf{a}_l}}$$ для рассчета $\color{red}{\dfrac{\partial \mathbf{a}_{l}}{\partial \mathbf{w}_{l}}}$ нам нужен только якобиан функции активации $l$-го слоя по параметрам слоя $\mathbf{w}_{l}$. 

* Градиент функции потерь по результатам рассчета функции активации слоя  $l$ может быть рассчитан рекурсивно по результатам слоя $l$, собственно тут и происходит __обратное распространение__:
$\color{blue}{\dfrac{\partial E}{\partial \mathbf{a}_l}}=\left ( \dfrac{\partial \mathbf{a}_{l+1}}{\partial \mathbf{a}_{l}} \right )^T \cdot \dfrac{\partial E}{\partial \mathbf{a}_{l+1}}=\left ( \color{magenta}{\dfrac{\partial \mathbf{a}_{l+1}}{\partial \mathbf{x}_{l+1}}} \right )^T \cdot \color{blue}{\dfrac{\partial E}{\partial \mathbf{a}_{l+1}}}$ для рассчета $\color{magenta}{\dfrac{\partial \mathbf{a}_{l+1}}{\partial \mathbf{x}_{l+1}}}$ нам нужен только якобиан функции активации $l+1$-го слоя по входным значениям слоя $\mathbf{x}_{l+1}$

* Т.е. чтобы проводить обратное распространение ошибки, нам на каждом слое (например $l$-м) нужно рассчитывать два якобиана:
    * якобиан функции активации $l$-го слоя по параметрам слоя $\color{red}{\dfrac{\partial \mathbf{a}_{l}}{\partial \mathbf{w}_{l}}}$ - он позволит рассчитать градиент $\dfrac{\partial E}{\partial \mathbf{w}_l}$ и сделать очередной шаг градиентного спуска для параметров этого слоя: $\mathbf{w}_l^{t+1} = \mathbf{w}_l^{t}-\gamma\nabla_{w_l} E(\mathbf{w}^{t})=\mathbf{w}_l^{t}-\gamma \dfrac{\partial E(\mathbf{w}^{t})}{\partial \mathbf{w}_l}$
    * якобиан функции активации $l$-го слоя по входным значениям слоя: $\color{magenta}{\dfrac{\partial \mathbf{a}_{l}}{\partial \mathbf{x}_{l}}}$ - он позволит распространить ошибку на низлежащие слои.

Такми образом при обучении нам нужна только __очень локальная информация, содержащаяся в самом слое__. Т.е.:
* __нет нобходимости знать как устроены сосоедние слои__: между слоями __очень простой интерфейс__ 
* т.е. __можно создать модульную архитектуру для слоев нейронной сети__: каждый модуль рассчитывает значение функции активации на основе выходов на прямом проходе и распространяет ошибку пришедшую на выходы на обратном проходе; все модули станадртным образом стыкуются друг с другом
* при модульной архитектуре граф нейронной сети может быть очень сложным, но его __рассчет выполняется по одной простой и универсальной схеме__
* внутри __модули могут быть сложно устроены, это никак не меняет логику остальных модулей и всего процесса обучения__, главное чтобы модуль корректно выполнял прямой и обратный проход.

-----------

##  Дифференцируемое программирование и реализация обратного распространения ошибки <a class="anchor" id="дифференцируемое"></a>
* [к оглавлению](#разделы)

__Почему Tensor *Flow*?__

<em class="qs"></em> Как реализовать __алгоритм обратного распространения ошибки__ удобно для использования в задачах моделирования ИНС?

Основная абстракция TensorFlow, PyTorch и других аналогичных библиотеках - __граф потока вычислений__.

* Рассматриваемые библиотеки обычно:
    1. задают __граф потока вычислений__ (формирует объект отложенных вычислений)
    2. запускают __процедуру выполненния отложенных вычислений__ и получает __результаты__ вычислений (в т.ч. ошибку модели).
* Возможность в явном виде работать с графом потока вычислений дает большое приемущество для __автоматического решения задачи обратного распространения ошибки__, являющейся составляющей адачей обучения модели ИНС.

<center> 
<img src="./img/ker_6.png" alt="Принцип устройства графа потока вычислений в TensorFlow" style="width: 400px;"/><br/>
    <b>Принцип устройства графа потока вычислений в TensorFlow</b>    
</center>

* Нейронная сеть это иерархия (она может быть простой и очень сложной) связанных (последовательно применяемых) функций слоев ИНС. Модель сети $f_L$ может быть представленна как суперпозиция из $L$ слоев $h^i\text{, }i \in \{1, \ldots, L\}$, каждый из которых параметризуется своими весами $w_i$:
$$f_L(\pmb{x}, \pmb{\theta})=f_L(\pmb{x}, \pmb{w}_1, \ldots, \pmb{w}_L )=h^L(h^{L-1}(\ldots h^1(\pmb{x}, \pmb{w}_1), \ldots, \pmb{w}_{L-1}),\pmb{w}_L)$$

* Вычисление функций слоев и взаимосвязи между слоями формируют граф потока вычислений в библиотеке моделирования ИНС.

<center> 
<img src="./img/ann_16.png" alt="Примеры иерархий в нейронных сетях" style="width: 400px;"/><br/>
    <b>Примеры иерархий в нейронных сетях</b>    
</center>


* По сути, ИНС это композиция модулей, представляющих собой слои нейронной сети:
    * если сеть прямого распространения (feedforward), то все просто
    * если сеть является направленным ациклическим графом, то существует правильный порядок применения функций
    * в случае, если есть циклы, образующие рекуррентные связи, то существуют специальные подходы (будут рассмотрены позднее)


* На обратном проходе (при обратном распространении ошибки) нам необходимо __дифференциировать сложную функцию__ многослойной ИНС
$$\nabla_\theta E(f_L(\mathbf{x}, \mathbf{\theta}), \mathbf{y})=\nabla_{\mathbf{w}_i} E(f_L(\mathbf{x}, \mathbf{w}_1, \ldots, \mathbf{w}_L ), \mathbf{y})=\nabla_{\mathbf{w}_i} E(h^L(h^{L-1}(\ldots h^1(\mathbf{x}, \mathbf{w}_1), \ldots, \mathbf{w}_{L-1}),\mathbf{w}_L), \mathbf{y})$$
* алгоритм обратного распространения ошибки позволяет свести эту задачу к дифференциированию составляющих функций, но для этого необходимо __храниить информацию о виде и взаимосвязях функций задействованных в расчете модели ИНС__, именно эта информация и хранится в графе потока вычислений. Это позволяет организовать __автоматическое дифференциирование__ сложной функци многослойной ИНС.

__Дифференциируемое программирование__

<em class="df"></em> __Дифференциируемое программирование__ (differentiable programming) - парадигма программирования при которой программа (функция рассчета значения) может быть продифференциирована в любой точке, обычно с помощью __автоматического диффиренциирования__. 

Это свойство позволяет использовать к программе __методы оптимизации основанные на рассчете градиента__, обычно - __методы градиентного спуска__.

Дифференциируемое программирование используется в:
* глубоком обучении
* глубоком обучении комбинированном с физическими моделями в робототехнике
* специализированных методах трассировки лучей
* обработке изображений

Большинство фреймоврков для дифференциируемого программирования использует граф потока вычислений определяющий выполнение программы и ее структуры данных.

Основные классы фреймворков для дифференциируемого программирования:
* __статические__ - они компилируют граф потока вычислений. Типичные представители: TensorFlow, Theano и др. Плюсы и минусы
    * <em class="pl"></em> могут использовать оптимизацию при компиляции
    * <em class="pl"></em> легче масштабирются на большие системы
    * <em class="mn"></em> статичность ограничевает интерактивность
    * <em class="mn"></em> многие программы не могут реализовываться легко (в частности: циклы, рекурсия)
* __динамические__ - динамически исполняют граф потока вычислений. Используют перегрузку операторов для записи. Типичные представители: PyTorch, AutoGradrFlow. Плюсы и минусы:
    * <em class="pl"></em> более простая и понятная запись программы
    * <em class="mn"></em> накладные расходы интерпретатора
    * <em class="mn"></em> невозможно использовать оптимизацию компилятора
    * <em class="mn"></em> хуже масштабируемость
* статическая на основе разбора промежуточного представления синтаксического разбора исходной программы. Пример фрэймоврк Zygote (язык программирования Julia).

__Прямой проход__:
* Модули из графа обходятся один за одним начиная с узла входных данных и далее по мере готовности всех необходимых входных данных для очередного модуля, который еще не был обойден
* Рассчет функций активации для каждого модуля по входным данным: $a_l=h_l(x_l, w_l)$
* Промежуточные значения кэшируются, чтобы не рассчитывать их повторно (в сложном графе сети и при обратном проходе)
* Выходы одних модулей становятся входами других модулей: $x_{l+1}=a_l$
* Последним модулем рассчитывается сумма потерь для входных данных
<center> 
    
__Прямой и обратный проход процедуры обучения многослойной ИНС:__

<img src="./img/ann_19.png" alt="Прямой и обратный проход " style="width: 300px;"/>
</center>


__Обратный проход__:
* Сначала должен быть произведен прямой проход. На входе обратного прохода известна сумма потерь.
* Строится обратный порядок обхода графа зависимостей модулей.
* Модули из графа обходятся один за одним начиная с узла рассчета функции потерь и далее по мере готовности всех необходимых входных данных для очередного модуля, который еще не был обойден 
* Для каждого модуля рассчитыватся якобиан функции активации по параметрам слоя $\color{red}{\dfrac{\partial \mathbf{a}_{l}}{\partial \mathbf{w}_{l}}}$ и якобиан функции активации по входным значениям слоя: $\color{magenta}{\dfrac{\partial \mathbf{a}_{l}}{\partial \mathbf{x}_{l}}}$ 
* По пришедшму в модуль градиенту ошибки (полученному из модулей использовавших результаты данного модуля на прямом проходе) $\color{blue}{\dfrac{\partial E}{\partial \mathbf{a}_l}}$ рассчитывается:
    * Градиент для шага градиентного спуска по параметрам модуля $w_l$: $\dfrac{\partial E}{\partial \mathbf{w}_l}=\left ( \color{red}{ \dfrac{\partial \mathbf{a}_{l}}{\partial \mathbf{w}_{l}}} \right )^T \cdot \color{blue}{\dfrac{\partial E}{\partial \mathbf{a}_l}}$
    * Градиент ошбки, который передается в модули, поставившие данные в этот модуль во время прямого прохода: $\color{blue}{\dfrac{\partial E}{\partial \mathbf{a}_{l-1}}}=\left ( \color{magenta}{\dfrac{\partial \mathbf{a}_{l}}{\partial \mathbf{x}_{l}}} \right )^T \cdot \color{blue}{\dfrac{\partial E}{\partial \mathbf{a}_{l}}}$ 

<center> 
    
__Пример: прямой проход, шаг 1__

<img src="./img/bp_2.png" alt="Пример" style="width: 700px;"/>
</center>

<center> 
    
__Пример: прямой проход, шаг 2__

<img src="./img/bp_3.png" alt="Пример" style="width: 700px;"/>
</center>

<center> 
    
__Пример: прямой проход, шаг 3__

<img src="./img/bp_4.png" alt="Пример" style="width: 700px;"/>
</center>

<center> 
    
__Пример: обратный проход, шаг 1__

<img src="./img/bp_5.png" alt="Пример" style="width: 700px;"/>
</center>

<center> 
    
__Пример: обратный проход, шаг 2__

<img src="./img/bp_6.png" alt="Пример" style="width: 700px;"/>
</center>

<center> 
    
__Производные популярных функций активации__

<img src="./img/ann_17.png" alt="Пример" style="width: 500px;"/>
</center>


<center> 
    
__Пример: обратный проход, шаг 3__

<img src="./img/bp_7.png" alt="Пример" style="width: 700px;"/>
</center>

:
##  Автоматическое дифференциирование в PyTorch <a class="anchor" id="автоматическое-PyTorch"></a>
* [к оглавлению](#разделы)

In [30]:
# The autograd package provides automatic differentiation 
# for all operations on Tensors

# requires_grad = True -> tracks all operations on the tensor. 
x = torch.randn(3, requires_grad=True)
y = x + 2

# y was created as a result of an operation, so it has a grad_fn attribute.
# grad_fn: references a Function that has created the Tensor
print(x) # created by the user -> grad_fn is None
print(y)
print(y.grad_fn)

tensor([-0.02651, -0.59082,  0.87850], requires_grad=True)
tensor([1.97349, 1.40918, 2.87850], grad_fn=<AddBackward0>)


In [31]:
# Do more operations on y
z = y * y * 3
print(z)
z = z.mean()
print(z)

tensor([11.68397,  5.95739, 24.85736], grad_fn=<MulBackward0>)
tensor(14.16624, grad_fn=<MeanBackward0>)


In [32]:
# Let's compute the gradients with backpropagation
# When we finish our computation we can call .backward() and have all the gradients computed automatically.
# The gradient for this tensor will be accumulated into .grad attribute.
# It is the partial derivate of the function w.r.t. the tensor

z.backward()
print(x.grad) # dz/dx

# Generally speaking, torch.autograd is an engine for computing vector-Jacobian product
# It computes partial derivates while applying the chain rule

tensor([3.94698, 2.81837, 5.75701])


---

__Примеры рассчета градиента суперпозиции двух функций нескольких переменных:__

<img src="./img/ann_18.png" alt="Примеры иерархий в нейронных сетях" style="width: 500px;"/>
</center>
Т.е. нам нужны градиенты по всем возможным путям (рассмотренным в обработном порядке) завимиостей переменных.

Запись этой же задачи в векторной нотации: 
* $\frac{\mathrm{d} z}{\mathrm{d} \mathbf{x}} = \nabla_x (z)= \begin{pmatrix}
    \dfrac{\partial z}{\partial x_1} \\ \cdots \\ \dfrac{\partial z}{\partial x_n} \end{pmatrix}=\left ( \frac{\mathrm{d} \mathbf{y}}{\mathrm{d} \mathbf{x}} \right )^T \cdot \nabla_y (z) = J(\mathbf{y}(\mathbf{x}))^T \cdot \nabla_y (z)= J(\mathbf{y}(\mathbf{x}))^T \cdot \begin{pmatrix}
    \dfrac{\partial z}{\partial y_1} \\ \cdots \\ \dfrac{\partial z}{\partial y_m} \end{pmatrix}$    
* Где $J$ это Якобиан: $$J(\mathbf{y}(\mathbf{x})) = \begin{pmatrix}
    \dfrac{\partial y_1}{\partial x_1} & \cdots & \dfrac{\partial y_1}{\partial x_n}\\
    \vdots & \ddots & \vdots\\
    \dfrac{\partial y_m}{\partial x_1} & \cdots & \dfrac{\partial y_m}{\partial x_n} \end{pmatrix} $$ 

In [33]:
# Model with non-scalar output:
# If a Tensor is non-scalar (more than 1 elements), we need to specify arguments for backward() 
# specify a gradient argument that is a tensor of matching shape.
# needed for vector-Jacobian product

x = torch.randn(3, requires_grad=True)

y = x * 2
for _ in range(10):
    y = y * 2

print(y)
print(y.shape)

tensor([ 1122.91479, -2130.42871,  3587.57251], grad_fn=<MulBackward0>)
torch.Size([3])


In [34]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float32)
y.backward(v)
print(x.grad)

tensor([2.04800e+02, 2.04800e+03, 2.04800e-01])


--- 
Stop a tensor from tracking history:
For example during our training loop when we want to update our weights
then this update operation should not be part of the gradient computation
* `x.requires_grad_(False)`
* `x.detach()`
* wrap in `with torch.no_grad():`

In [35]:
# .requires_grad_(...) changes an existing flag in-place.

a = torch.randn(2, 2)
print(f'a.requires_grad = {a.requires_grad}')

b = ((a * 3) / (a - 1))
print(f'b.grad_fn = {b.grad_fn}')
      
a.requires_grad_(True)
print(f'a.requires_grad = {a.requires_grad}')

b = (a * a).sum()
print(f'b.grad_fn = {b.grad_fn}')

a.requires_grad = False
b.grad_fn = None
a.requires_grad = True
b.grad_fn = <SumBackward0 object at 0x0000025E9219B370>


In [36]:
# .detach(): get a new Tensor with the same content but no gradient computation:
a = torch.randn(2, 2, requires_grad=True)
print(a.requires_grad)

b = a.detach()
print(b.requires_grad)

True
False


In [37]:
# wrap in 'with torch.no_grad():'
a = torch.randn(2, 2, requires_grad=True)
print(a.requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
False


------

__Пример использования градиентного спуска с автоматическим диффиринциированием в PyTorch:__

In [40]:
# backward() accumulates the gradient for this tensor into .grad attribute.
# !!! We need to be careful during optimization !!!
# Use .zero_() to empty the gradients before a new optimization step!

weights = torch.ones(4, requires_grad=True)

for epoch in range(3):
    # just a dummy example
    # 'forward pass'
    model_output = (weights*3).sum()
    
    
    model_output.backward()
    
    print(f'ep: {epoch}, weights: {weights}, weights.grad: {weights.grad}, m_output: {model_output:.3f}')

    # optimize model, i.e. adjust weights...
    with torch.no_grad():
        weights -= 0.1 * weights.grad

    # this is important! It affects the final weights & output
    weights.grad.zero_()
    
print(f'ep: {epoch} (FIN), weights: {weights}, weights.grad: {weights.grad}, m_output: {model_output:.3f}')    

# Optimizer has zero_grad() method
# optimizer = torch.optim.SGD([weights], lr=0.1)
# During training:
# optimizer.step()
# optimizer.zero_grad()

ep: 0, weights: tensor([1., 1., 1., 1.], requires_grad=True), weights.grad: tensor([3., 3., 3., 3.]), m_output: 12.000
ep: 1, weights: tensor([0.70000, 0.70000, 0.70000, 0.70000], requires_grad=True), weights.grad: tensor([3., 3., 3., 3.]), m_output: 8.400
ep: 2, weights: tensor([0.40000, 0.40000, 0.40000, 0.40000], requires_grad=True), weights.grad: tensor([3., 3., 3., 3.]), m_output: 4.800
ep: 2 (FIN), weights: tensor([0.10000, 0.10000, 0.10000, 0.10000], requires_grad=True), weights.grad: tensor([0., 0., 0., 0.]), m_output: 4.800


-----

Автоматическое выполнение обратного прохода с помощью `l.backward()`:

In [41]:
# Модель линейной регрессии (с несколькими параметрами)
# f = X * w 

# Данные для обучения: 
# принзаки X: рассматривается 4 наблюдения (ось 0) и 2 признака (ось 1):

X = torch.tensor([[1., 40.],
                  [2., 30.],
                  [3., 20.],
                  [4., 10.]], dtype=torch.float32) # Size([4, 2])


# истинное значение весов (используется только для получения обучающих правильных ответов):
w_ans = torch.tensor([2., 1.], dtype=torch.float32)
# Y - приавильные ответы: 
Y = X @ w_ans

torch.set_printoptions(precision=5) # точность вывода на печать значений тензоров
print(f'w true value = {w_ans}, Y = {Y}')

# model (модель, в нашем случае: линейная регрессия)

# изначальное значение весов w
#!!! requires_grad=True
w = torch.tensor([0.0, 0.0], dtype=torch.float32, requires_grad=True)

# прямое распространение:
def forward(X):
    return X @ w # Size([4])

# loss = MSE (функция потерь, в нашем слаучае: средняя квадратичная ошибка)
def loss(y, y_pred):
    return ((y_pred - y)**2).mean() # Size([])


# Training
learning_rate = 0.0013
n_iters = 1000 + 1

# основной цикл:
for epoch in range(n_iters):
    # predict = forward pass
    y_pred = forward(X)

    # loss
    l = loss(Y, y_pred)
    
    #!!! backward pass        
    # calculate gradients = backward pass
    l.backward()
    
    # update weights
    #w.data = w.data - learning_rate * w.grad
    with torch.no_grad():
        w -= learning_rate * w.grad

    if epoch % 5 == 0:
        dw = w.grad
        print(f'ep {epoch}: w = {w}, loss = {l:.8f} grad = {dw}')                        
#         print(f'ep {epoch}: w = {w}, y_p = {y_pred}, loss = {l:.8f}\ngrad = {dw}')                
#         print(f'ep {epoch:3}: y_p = {y_pred}, loss = {l:.8f} grd = {dw}')
                
        
    # zero the gradients after updating
    w.grad.zero_()    
        

w true value = tensor([2., 1.]), Y = tensor([42., 34., 26., 18.])
ep 0: w = tensor([0.16900, 2.21000], requires_grad=True), loss = 980.00000000 grad = tensor([ -130., -1700.])
ep 5: w = tensor([0.13813, 0.24422], requires_grad=True), loss = 646.58898926 grad = tensor([  77.23860, 1378.76135])
ep 10: w = tensor([0.33966, 1.82453], requires_grad=True), loss = 427.49307251 grad = tensor([  -89.12965, -1114.91821])
ep 15: w = tensor([0.34364, 0.53338], requires_grad=True), loss = 283.42614746 grad = tensor([ 47.01924, 904.69263])
ep 20: w = tensor([0.49880, 1.56850], requires_grad=True), loss = 188.61228943 grad = tensor([ -61.92347, -731.13928])
ep 25: w = tensor([0.52316, 0.72007], requires_grad=True), loss = 126.13926697 grad = tensor([ 27.57067, 593.68494])
ep 30: w = tensor([0.64554, 1.39771], requires_grad=True), loss = 84.91012573 grad = tensor([ -43.72146, -479.40924])
ep 35: w = tensor([0.68103, 0.83984], requires_grad=True), loss = 57.64197159 grad = tensor([ 15.14910, 389.64639]

ep 360: w = tensor([1.96161, 1.00257], requires_grad=True), loss = 0.00627505 grad = tensor([-0.32198,  0.02117])
ep 365: w = tensor([1.96364, 1.00244], requires_grad=True), loss = 0.00563018 grad = tensor([-0.30494,  0.02073])
ep 370: w = tensor([1.96556, 1.00231], requires_grad=True), loss = 0.00505164 grad = tensor([-0.28889,  0.01907])
ep 375: w = tensor([1.96737, 1.00219], requires_grad=True), loss = 0.00453248 grad = tensor([-0.27360,  0.01865])
ep 380: w = tensor([1.96910, 1.00207], requires_grad=True), loss = 0.00406663 grad = tensor([-0.25919,  0.01716])
ep 385: w = tensor([1.97073, 1.00196], requires_grad=True), loss = 0.00364873 grad = tensor([-0.24547,  0.01686])
ep 390: w = tensor([1.97227, 1.00186], requires_grad=True), loss = 0.00327375 grad = tensor([-0.23257,  0.01525])
ep 395: w = tensor([1.97374, 1.00176], requires_grad=True), loss = 0.00293732 grad = tensor([-0.22025,  0.01501])
ep 400: w = tensor([1.97512, 1.00167], requires_grad=True), loss = 0.00263548 grad = ten

ep 745: w = tensor([1.99941, 1.00004], requires_grad=True), loss = 0.00000149 grad = tensor([-0.00494,  0.00058])
ep 750: w = tensor([1.99944, 1.00004], requires_grad=True), loss = 0.00000133 grad = tensor([-0.00467,  0.00057])
ep 755: w = tensor([1.99947, 1.00004], requires_grad=True), loss = 0.00000119 grad = tensor([-0.00445,  0.00010])
ep 760: w = tensor([1.99950, 1.00003], requires_grad=True), loss = 0.00000107 grad = tensor([-0.00419,  0.00046])
ep 765: w = tensor([1.99952, 1.00003], requires_grad=True), loss = 0.00000096 grad = tensor([-0.00398,  0.00027])
ep 770: w = tensor([1.99955, 1.00003], requires_grad=True), loss = 0.00000086 grad = tensor([-0.00379,  0.00010])
ep 775: w = tensor([1.99957, 1.00003], requires_grad=True), loss = 0.00000077 grad = tensor([-0.00357,  0.00038])
ep 780: w = tensor([1.99960, 1.00003], requires_grad=True), loss = 0.00000069 grad = tensor([-3.39699e-03,  6.67572e-05])
ep 785: w = tensor([1.99962, 1.00003], requires_grad=True), loss = 0.00000062 gr

------
Использование встроенного оптимизатора `optimizer = torch.optim.SGD([w], lr=learning_rate)` и функции потерь `loss = nn.MSELoss()`:

In [42]:
import torch
import torch.nn as nn

# Модель линейной регрессии (с несколькими параметрами)
# f = X * w 

#--------------------
# 0) Training samples

# Данные для обучения: 
# принзаки X: рассматривается 4 наблюдения (ось 0) и 2 признака (ось 1):
X = torch.tensor([[1., 40.],
                  [2., 30.],
                  [3., 20.],
                  [4., 10.]], dtype=torch.float32) # Size([4, 2])

# истинное значение весов (используется только для получения обучающих правильных ответов):
w_ans = torch.tensor([2., 1.], dtype=torch.float32)
# Y - приавильные ответы: 
Y = X @ w_ans

torch.set_printoptions(precision=5) # точность вывода на печать значений тензоров
print(f'w true value = {w_ans}, Y = {Y}')

#--------------------
# 1) Design Model: Weights to optimize and forward function

# изначальное значение весов w
w = torch.tensor([0.0, 0.0], dtype=torch.float32, requires_grad=True)


# model (модель, в нашем случае: линейная регрессия)
# прямое распространение:
def forward(X):
    return X @ w # Size([4])

#--------------------
# 2) Define loss and optimizer

# callable function
loss = nn.MSELoss()

# loss = MSE (функция потерь, в нашем слаучае: средняя квадратичная ошибка)
# def loss(y, y_pred):
#     return ((y_pred - y)**2).mean() # Size([])

learning_rate = 0.0013
optimizer = torch.optim.SGD([w], lr=learning_rate)


#--------------------
# 3) Training loop
# основной цикл:
n_iters = 1000 + 1

for epoch in range(n_iters):
    # predict = forward pass
    y_pred = forward(X)

    # loss
    l = loss(Y, y_pred)
         
    # calculate gradients = backward pass
    l.backward()
    
    # update weights
    optimizer.step()
    
    if epoch % 5 == 0:
        dw = optimizer.param_groups[0]['params'][0].grad
        print(f'ep {epoch}: w = {w}, loss = {l:.8f} grad = {w.grad}')                        
#         print(f'ep {epoch}: w = {w}, y_p = {y_pred}, loss = {l:.8f}\ngrad = {dw}')                
#         print(f'ep {epoch:3}: y_p = {y_pred}, loss = {l:.8f} grd = {dw}')    

    # zero the gradients after updating
    optimizer.zero_grad()    
        

w true value = tensor([2., 1.]), Y = tensor([42., 34., 26., 18.])
ep 0: w = tensor([0.16900, 2.21000], requires_grad=True), loss = 980.00000000 grad = tensor([ -130., -1700.])
ep 5: w = tensor([0.13813, 0.24422], requires_grad=True), loss = 646.58898926 grad = tensor([  77.23860, 1378.76135])
ep 10: w = tensor([0.33966, 1.82453], requires_grad=True), loss = 427.49307251 grad = tensor([  -89.12965, -1114.91821])
ep 15: w = tensor([0.34364, 0.53338], requires_grad=True), loss = 283.42614746 grad = tensor([ 47.01924, 904.69263])
ep 20: w = tensor([0.49880, 1.56850], requires_grad=True), loss = 188.61228943 grad = tensor([ -61.92347, -731.13928])
ep 25: w = tensor([0.52316, 0.72007], requires_grad=True), loss = 126.13926697 grad = tensor([ 27.57067, 593.68494])
ep 30: w = tensor([0.64554, 1.39771], requires_grad=True), loss = 84.91012573 grad = tensor([ -43.72146, -479.40924])
ep 35: w = tensor([0.68103, 0.83984], requires_grad=True), loss = 57.64197159 grad = tensor([ 15.14910, 389.64639]

ep 370: w = tensor([1.96556, 1.00231], requires_grad=True), loss = 0.00505164 grad = tensor([-0.28889,  0.01907])
ep 375: w = tensor([1.96737, 1.00219], requires_grad=True), loss = 0.00453248 grad = tensor([-0.27360,  0.01865])
ep 380: w = tensor([1.96910, 1.00207], requires_grad=True), loss = 0.00406663 grad = tensor([-0.25919,  0.01716])
ep 385: w = tensor([1.97073, 1.00196], requires_grad=True), loss = 0.00364873 grad = tensor([-0.24547,  0.01686])
ep 390: w = tensor([1.97227, 1.00186], requires_grad=True), loss = 0.00327375 grad = tensor([-0.23257,  0.01525])
ep 395: w = tensor([1.97374, 1.00176], requires_grad=True), loss = 0.00293732 grad = tensor([-0.22025,  0.01501])
ep 400: w = tensor([1.97512, 1.00167], requires_grad=True), loss = 0.00263548 grad = tensor([-0.20865,  0.01399])
ep 405: w = tensor([1.97643, 1.00158], requires_grad=True), loss = 0.00236459 grad = tensor([-0.19764,  0.01316])
ep 410: w = tensor([1.97768, 1.00150], requires_grad=True), loss = 0.00212172 grad = ten

ep 740: w = tensor([1.99938, 1.00004], requires_grad=True), loss = 0.00000166 grad = tensor([-5.25570e-03, -3.81470e-05])
ep 745: w = tensor([1.99941, 1.00004], requires_grad=True), loss = 0.00000149 grad = tensor([-0.00494,  0.00058])
ep 750: w = tensor([1.99944, 1.00004], requires_grad=True), loss = 0.00000133 grad = tensor([-0.00467,  0.00057])
ep 755: w = tensor([1.99947, 1.00004], requires_grad=True), loss = 0.00000119 grad = tensor([-0.00445,  0.00010])
ep 760: w = tensor([1.99950, 1.00003], requires_grad=True), loss = 0.00000107 grad = tensor([-0.00419,  0.00046])
ep 765: w = tensor([1.99952, 1.00003], requires_grad=True), loss = 0.00000096 grad = tensor([-0.00398,  0.00027])
ep 770: w = tensor([1.99955, 1.00003], requires_grad=True), loss = 0.00000086 grad = tensor([-0.00379,  0.00010])
ep 775: w = tensor([1.99957, 1.00003], requires_grad=True), loss = 0.00000077 grad = tensor([-0.00357,  0.00038])
ep 780: w = tensor([1.99960, 1.00003], requires_grad=True), loss = 0.00000069 gr

-----------
Использование модели:

`torch.nn.Linear(in_features, out_features, bias=True)`
Applies a linear transformation to the incoming data: $y = xA^T + b$

Parameters:
* `in_features` – size of each input sample
* `out_features` – size of each output sample
* `bias` – If set to False, the layer will not learn an additive bias. Default: True

In [43]:
# X_test = torch.tensor([[1], [2], [3], [4]], dtype=torch.float32) # torch.tensor([5], dtype=torch.float32)

X = torch.tensor([[1., 40.],
                  [2., 30.],
                  [3., 20.],
                  [4., 10.]], dtype=torch.float32) # Size([4, 2])

n_samples, n_features = X.shape

print(n_samples, n_features)

# n_samples, n_features = X_test.shape
# input_size = n_features
# output_size = n_features

class LinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        # define diferent layers
        self.lin = nn.Linear(input_dim, output_dim, bias=False)
    def forward(self, x):
        return self.lin(x)
    
model = LinearRegression(n_features, 1)


print(f'Prediction before training: f(5) = {model(X)}')

4 2
Prediction before training: f(5) = tensor([[-18.99730],
        [-14.79363],
        [-10.58997],
        [ -6.38630]], grad_fn=<MmBackward0>)


In [82]:
# X_test.shape

In [44]:
import torch
import torch.nn as nn

# Модель линейной регрессии (с несколькими параметрами)
# f = X * w 

#--------------------
# 0) Training samples

# Данные для обучения: 
# принзаки X: рассматривается 4 наблюдения (ось 0) и 2 признака (ось 1):
X = torch.tensor([[1., 40.],
                  [2., 30.],
                  [3., 20.],
                  [4., 10.]], dtype=torch.float32) # Size([4, 2])

print(f'X.shape = {X.shape}')
X_samples, X_features = X.shape

# истинное значение весов (используется только для получения обучающих правильных ответов):
w_ans = torch.tensor([2., 1.], dtype=torch.float32)
# Y - приавильные ответы: 
Y = X @ w_ans
Y = Y.reshape((-1, 1))
print(f'Y.shape = {Y.shape}')
Y_features = 1

torch.set_printoptions(precision=5) # точность вывода на печать значений тензоров
print(f'w true value = {w_ans}, Y = {Y}')

#--------------------
# 1) Design Model, the model has to implement the forward pass!
# Here we can use a built-in model from PyTorch
input_size = n_features
output_size = n_features

# we can call this model with samples X
# model = nn.Linear(input_size, output_size)

class LinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        # define diferent layers
        self.lin = nn.Linear(input_dim, output_dim, bias=False)
    def forward(self, x):
        return self.lin(x)
    
model = LinearRegression(X_features, Y_features)


print(f'Prediction before training: f({X}) = {model(X)}\n w={model.lin.weight.data}')


# # model (модель, в нашем случае: линейная регрессия)
# # прямое распространение:
# def forward(X):
#     return X @ w # Size([4])

#--------------------
# 2) Define loss and optimizer

# callable function
criterion  = nn.MSELoss()

learning_rate = 0.0013
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

#--------------------
# 3) Training loop
# основной цикл:
n_iters = 1000 + 1
# n_iters = 40 + 1

for epoch in range(n_iters):
    # Forward pass and loss
    y_predicted = model(X)
    loss = criterion(y_predicted, Y)

    
    # Backward pass and update
    loss.backward()
    optimizer.step()

    if epoch % 5 == 0:
        w = model.lin.weight.data
        dw = model.lin.weight.grad
        print(f'ep {epoch}: w = {w}, loss = {loss:.8f} grad = {dw}')                        
#         print(f'ep {epoch}: w = {w}, y_p = {y_pred}, loss = {l:.8f}\ngrad = {dw}')                
#         print(f'ep {epoch:3}: y_p = {y_pred}, loss = {l:.8f} grd = {dw}')       
    
    # zero grad before new step
    optimizer.zero_grad()    
    
#     if epoch % 5 == 0:
#         print(f'epoch {epoch}: w = {w}, y_pred = {y_pred}, loss = {l:.8f}\ngradient = {dw}')
    

X.shape = torch.Size([4, 2])
Y.shape = torch.Size([4, 1])
w true value = tensor([2., 1.]), Y = tensor([[42.],
        [34.],
        [26.],
        [18.]])
Prediction before training: f(tensor([[ 1., 40.],
        [ 2., 30.],
        [ 3., 20.],
        [ 4., 10.]])) = tensor([[10.77437],
        [ 8.85886],
        [ 6.94336],
        [ 5.02785]], grad_fn=<MmBackward0>)
 w=tensor([[0.62247, 0.25380]])
ep 0: w = tensor([[0.74634, 1.88797]]), loss = 534.63720703 grad = tensor([[  -95.28321, -1257.05676]])
ep 5: w = tensor([[0.71822, 0.43480]]), loss = 352.47018433 grad = tensor([[  57.90713, 1019.41907]])
ep 10: w = tensor([[0.86223, 1.60364]]), loss = 232.79104614 grad = tensor([[ -65.14955, -824.43488]])
ep 15: w = tensor([[0.86043, 0.64926]]), loss = 154.12103271 grad = tensor([[ 35.48081, 668.89294]])
ep 20: w = tensor([[0.97066, 1.41495]]), loss = 102.36933899 grad = tensor([[ -45.11044, -540.65851]])
ep 25: w = tensor([[0.98441, 0.78789]]), loss = 68.29065704 grad = tensor([[ 21.0

ep 420: w = tensor([[1.98625, 1.00092]]), loss = 0.00080559 grad = tensor([[-0.11536,  0.00770]])
ep 425: w = tensor([[1.98697, 1.00087]]), loss = 0.00072281 grad = tensor([[-0.10926,  0.00740]])
ep 430: w = tensor([[1.98766, 1.00083]]), loss = 0.00064850 grad = tensor([[-0.10350,  0.00690]])
ep 435: w = tensor([[1.98831, 1.00078]]), loss = 0.00058183 grad = tensor([[-0.09804,  0.00648]])
ep 440: w = tensor([[1.98893, 1.00074]]), loss = 0.00052207 grad = tensor([[-0.09287,  0.00618]])
ep 445: w = tensor([[1.98951, 1.00070]]), loss = 0.00046841 grad = tensor([[-0.08797,  0.00585]])
ep 450: w = tensor([[1.99007, 1.00067]]), loss = 0.00042026 grad = tensor([[-0.08331,  0.00565]])
ep 455: w = tensor([[1.99059, 1.00063]]), loss = 0.00037706 grad = tensor([[-0.07893,  0.00514]])
ep 460: w = tensor([[1.99109, 1.00060]]), loss = 0.00033833 grad = tensor([[-0.07474,  0.00531]])
ep 465: w = tensor([[1.99156, 1.00057]]), loss = 0.00030357 grad = tensor([[-0.07083,  0.00449]])
ep 470: w = tensor([

ep 855: w = tensor([[1.99988, 1.00001]]), loss = 0.00000006 grad = tensor([[-0.00100,  0.00056]])
ep 860: w = tensor([[1.99988, 1.00001]]), loss = 0.00000006 grad = tensor([[-0.00099, -0.00019]])
ep 865: w = tensor([[1.99989, 1.00001]]), loss = 0.00000005 grad = tensor([[-0.00091,  0.00034]])
ep 870: w = tensor([[1.99990, 1.00001]]), loss = 0.00000005 grad = tensor([[-0.00092, -0.00057]])
ep 875: w = tensor([[1.99990, 1.00001]]), loss = 0.00000004 grad = tensor([[-0.00081,  0.00034]])
ep 880: w = tensor([[1.99991, 1.00001]]), loss = 0.00000004 grad = tensor([[-0.00077,  0.00023]])
ep 885: w = tensor([[1.99991, 1.00001]]), loss = 0.00000003 grad = tensor([[-0.00074,  0.00015]])
ep 890: w = tensor([[1.99992, 1.00001]]), loss = 0.00000003 grad = tensor([[-7.16209e-04, -8.58307e-05]])
ep 895: w = tensor([[1.99992, 1.00001]]), loss = 0.00000003 grad = tensor([[-0.00065,  0.00025]])
ep 900: w = tensor([[1.99992, 1.00001]]), loss = 0.00000002 grad = tensor([[-6.31332e-04,  6.67572e-05]])
ep 9

In [126]:
Y

tensor([42., 34., 26., 18.])

In [127]:
y_predicted

tensor([[26.74803],
        [22.73413],
        [18.72023],
        [14.70633]], grad_fn=<MmBackward0>)

In [128]:
model.lin.weight.data

tensor([[2.21181, 1.08752]])

In [131]:
model.lin.weight.data @ X.T

tensor([[45.71261, 37.04921, 28.38582, 19.72242]])

In [132]:
model.lin(x)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x3 and 2x1)

In [116]:
model.lin.weight.grad

tensor([[0., 0.]])

In [118]:
model.lin.weight.data

tensor([[5.99989, 0.60001]])

In [113]:
list(model.parameters())[0].data

tensor([[5.99989, 0.60001]])

In [112]:
type(list(model.parameters())[0])

torch.nn.parameter.Parameter

In [102]:
optimizer.param_groups

[{'params': [Parameter containing:
   tensor([[5.99989, 0.60001]], requires_grad=True)],
  'lr': 0.0013,
  'momentum': 0,
  'dampening': 0,
  'weight_decay': 0,
  'nesterov': False,
  'maximize': False,
  'foreach': None,
  'differentiable': False}]

In [108]:
optimizer.param_groups[0]['params'][0]

Parameter containing:
tensor([[5.99989, 0.60001]], requires_grad=True)

In [ ]:
optimizer.param_groups[0]['params'][0].grad

In [90]:
model.get_parameter()

TypeError: get_parameter() missing 1 required positional argument: 'target'

In [100]:
list(model.parameters())[0].data

tensor([[5.99988, 0.60001]])

In [97]:
for p in model.parameters():
#     p.requires_grad: bool
    print(p.data) #: Tensor

tensor([[5.99988, 0.60001]])


In [95]:
model.lin

Linear(in_features=2, out_features=1, bias=False)

In [92]:
dir(model.lin.parameters)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__func__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [88]:
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_backward_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_ver

---
# Спасибо за внимание!

---
### Технический раздел:

<br/> next <em class="qs"></em> qs line 
<br/> next <em class="an"></em> an line 
<br/> next <em class="nt"></em> an line 
<br/> next <em class="df"></em> df line 
<br/> next <em class="ex"></em> ex line 
<br/> next <em class="pl"></em> pl line 
<br/> next <em class="mn"></em> mn line 
<br/> next <em class="plmn"></em> plmn line 
<br/> next <em class="hn"></em> hn line 

* Работа с графом потока вычислений нужна  для того, чтобы решить __задачу обучения многослойной ИНС__. А эта задача требует после получения резуьтатов и оценки ошибки __выполнения обратного прохода__ дающего градиент ошибки для весов (параметров) модели и последующей процедуры оптимизации весов. 